**Intro**

In the following notebook, I will be creating a churn predictor for our bank 

**Read in libraries**

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**Set Notebook Preferences**

In [36]:
#Set plot style
plt.style.use('Solarize_Light2')

#Set path to draw_data
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Bank Churn Analysis\Data\02_Cleaned_Data'

**Read in data**

In [37]:
df = pd.read_csv(path + '/2020_0720_Cleaned_Churn_Date.csv', index_col=0)

## Data Overview

**Preview data**

In [38]:
#Display data shape and head
print('Data shape:', df.shape)
display(df.head())


Data shape: (10000, 11)


creditscore geography  gender  age  tenure    balance  numofproducts  \
0          619    France       0   42       2       0.00              1   
1          608     Spain       0   41       1   83807.86              1   
2          502    France       0   42       8  159660.80              3   
3          699    France       0   39       1       0.00              2   
4          850     Spain       0   43       2  125510.82              1   

   hascrcard  isactivemember  estimatedsalary  exited  
0          1               1        101348.88       1  
1          0               1        112542.58       0  
2          1               0        113931.57       1  
3          0               0         93826.63       0  
4          1               1         79084.10       0

**About the data - Info**

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   creditscore      10000 non-null  int64  
 1   geography        10000 non-null  object 
 2   gender           10000 non-null  int64  
 3   age              10000 non-null  int64  
 4   tenure           10000 non-null  int64  
 5   balance          10000 non-null  float64
 6   numofproducts    10000 non-null  int64  
 7   hascrcard        10000 non-null  int64  
 8   isactivemember   10000 non-null  int64  
 9   estimatedsalary  10000 non-null  float64
 10  exited           10000 non-null  int64  
dtypes: float64(2), int64(8), object(1)
memory usage: 937.5+ KB


**About the data - Description Statistics**

In [40]:
df.describe()

creditscore        gender           age        tenure        balance  \
count  10000.000000  10000.000000  10000.000000  10000.000000   10000.000000   
mean     650.528800      0.545700     38.921800      5.012800   76485.889288   
std       96.653299      0.497932     10.487806      2.892174   62397.405202   
min      350.000000      0.000000     18.000000      0.000000       0.000000   
25%      584.000000      0.000000     32.000000      3.000000       0.000000   
50%      652.000000      1.000000     37.000000      5.000000   97198.540000   
75%      718.000000      1.000000     44.000000      7.000000  127644.240000   
max      850.000000      1.000000     92.000000     10.000000  250898.090000   

       numofproducts    hascrcard  isactivemember  estimatedsalary  \
count   10000.000000  10000.00000    10000.000000     10000.000000   
mean        1.530200      0.70550        0.515100    100090.239881   
std         0.581654      0.45584        0.499797     57510.492818   
min         1.000000      0.00000        0.000000        11.580000   
25%         1.000000      0.00000        0.000000     51002.110000   
50%         1.000000      1.00000        1.000000    100193.915000   
75%         2.000000      1.00000        1.000000    149388.247500   
max         4.000000      1.00000        1.000000    199992.480000   

             exited  
count  10000.000000  
mean       0.203700  
std        0.402769  
min        0.000000  
25%        0.000000  
50%        0.000000  
75%        0.000000  
max        1.000000

# Machine Learning

## Prepare data

**Split data**

In [41]:
#Read in library
from sklearn.model_selection import train_test_split

#Seperate train/test data from target
X = df.drop('exited', axis = 1)
y = df.exited.values

#Split into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size =0.2, shuffle = False )

#Check
print('Training data shape:{} Label shape:{}'.format(X_train.shape, y_train.shape))
print('Test data shape:{} Label shape:{}'.format(X_test.shape, y_test.shape))

Training data shape:(8000, 10) Label shape:(8000,)
Test data shape:(2000, 10) Label shape:(2000,)


## Develop Preprocessing Pipeline

In [55]:
#Read in libraries
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_predict

#Get names of numeric/categorical features
num_features = X_train.select_dtypes(include = ['int64', 'float64']).columns
cat_features = X_train.select_dtypes(include = ['object']).columns

#Init steps for column transformera
num_transformer = Pipeline([('scaler', MinMaxScaler())])
cat_transformer = Pipeline([('onehot', OneHotEncoder(handle_unknown='ignore'))])

#Build preprecessor
preprocessor = ColumnTransformer(transformers=[('numerics', num_transformer, num_features),
                                              ('categoricals', cat_transformer, cat_features)],
                                n_jobs=-1)

#Build initial pipeline
classifier = Pipeline([('Preprocessor', preprocessor)])

## Build Base Model - Naive Bayes Classifier

**Build base model and get predictions**

In [56]:
#Import naive bayes
from sklearn.naive_bayes import GaussianNB

#Append GaussianNB to pipeline
classifier.steps.append(['model', GaussianNB()])

#Get predictions on training data
predictions = cross_val_predict(classifier, X_train, y_train,  n_jobs=-1)

**Evaluate base model**

In [60]:
#Import classification report
from sklearn.metrics import classification_report

#Evaluate
print('Naive Bayes Classification Report: \n',classification_report(y_train, predictions))

Naive Bayes Classification Report: 
               precision    recall  f1-score   support

           0       0.85      0.94      0.89      6353
           1       0.60      0.37      0.46      1647

    accuracy                           0.82      8000
   macro avg       0.73      0.65      0.68      8000
weighted avg       0.80      0.82      0.80      8000



## Model Selection

Evaluate a series of additional models to later tune into final classifier.

Goal is to optimize recall, we do not want to mislabel people that will probably be leaving so we have a chance to intervene and keep them.

In [61]:
#Read in classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

#Read in metrics
metrics.accuracy_score
metrics.precision_score
metrics.recall_score

#Init classifiers
classifiers = [KNeighborsClassifier(n_jobs=-1),
LogisticRegression(n_jobs=-1),
DecisionTreeClassifier(max_depth=10),
RandomForestClassifier(max_depth=10, n_jobs=-1)]

#Init model names
names= ['KNeighborsClassifier',
        'LogisticRegression',
'DecisionTreeClassifier',
'RandomForestClassifier'
    ]

In [ ]:
#Write a for-loop that fits each model to the data and evaluated recall
for name, model in zip(names, models):
    classifier.steps.pop(1) #Delete last step of pipe
    classifier.steps.append([name,model]) # append new step of pipe
    

In [64]:
classifier.steps[1]

['model', GaussianNB()]